In [1]:
import sys # for automation and parallelisation
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])

In [2]:
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.messaging as message
from biogeme import expressions as ex
import pandas as pd
import numpy as np
import xlsxwriter

# Calibration
## Estimation of the model parameters
quetzal_germany is being estimated using [PandasBiogeme](https://biogeme.epfl.ch/). This notebook estimates calibration parameters for the model's utility functions.
- Documentation and reference: [Bierlaire, M. (2020). A short introduction to PandasBiogeme. Technical report TRANSP-OR 200605. Transport and Mobility Laboratory, ENAC, EPFL.](https://transp-or.epfl.ch/documents/technicalReports/Bier20.pdf)
- Tutorial: https://www.youtube.com/watch?v=OiM94B8WayA

### Model formulation
The model consists of systematic utility functions, one for each mode. They comprise an alternaive-specific constant (ASC), a distance-dependent part with travel time and cost, and a cost damping function F. Additionally, person-specific attributes $x_p$ apply.

$V_{im} = ASC_{im} + F(T_m, b_{c_i}) + C_{m_i} b_{c_i} + \sum_{p \in P} x_p * b_{x_p}$

Index i marks the demand group

In [3]:
input_path = '../input/'
model_path = '../model/'

In [ ]:
# Load parameters for settings
params = excel.read_var(file='../input/parameters.xls', scenario=scenario)

### Prepare the database

In [4]:
df = pd.read_csv(input_path + 'transport_demand/calibration_inter-cellular_trips_MiD2017.csv')
df = df[['cost_rail_short', 'cost_rail_long', 'cost_car', 'cost_coach', 'cost_bus', 'cost_walk', 'cost_air',
         'time_rail_short', 'time_rail_long', 'time_car', 'time_coach', 'time_bus', 'time_walk', 'time_air',
         'cost_rail', 'cost_road', 'time_rail', 'time_road',
         'mode_model', 'purpose_vp', 'car_avail', 'length', 'W_GEW',
         'ticket_type', 'income_hh', 'cars_hh', 'education', 'license',
         'bike_avail', 'car_sharing_abo', 'car_dens_zone']]
df.columns = ['C_RAIL_S', 'C_RAIL_L', 'C_CAR', 'C_COACH', 'C_BUS', 'C_NON_MOTOR', 'C_AIR',
              'T_RAIL_S', 'T_RAIL_L', 'T_CAR', 'T_COACH', 'T_BUS', 'T_NON_MOTOR', 'T_AIR',
              'C_RAIL', 'C_ROAD', 'T_RAIL', 'T_ROAD',
              'MODE', 'PURPOSE', 'CAR_AV', 'DIST', 'W_GEW',
              'ABO', 'INCOME', 'CARS', 'EDU', 'DL', 'BIKE_AV', 'CS', 'CAR_DENS']

In [ ]:
# The estimation requires numerical purpose values
p_dict = {'commuting':1, 'business':2, 'education':3, 'buy/execute':4, 'leisure':6, 'accompany':7}
df['PURPOSE'] = df['PURPOSE'].apply(lambda s: p_dict[s.split('_')[0]])

In [5]:
inf = 10000
df = df.replace({np.inf:inf})

In [6]:
# "Normalise" PT ticket types
# Assume this dict with factors
ticket_dict = {1:0, 2:0.1, 3:0.5, 4:1, 5:1, 6:0, 7:0, 99:0, 200:0}
df['ABO'] = df['ABO'].map(ticket_dict)

In [9]:
# Scale time to hours
df['T_RAIL_Sc'] = df['T_RAIL'] / 60
df['T_RAIL_S_S'] = df['T_RAIL_S'] / 60
df['T_RAIL_L_S'] = df['T_RAIL_L'] / 60
df['T_CAR_S'] = df['T_CAR'] / 60
df['T_COACH_S'] = df['T_COACH'] / 60
df['T_BUS_S'] = df['T_BUS'] / 60
df['T_ROAD_S'] = df['T_ROAD'] / 60
df['T_AIR_S'] = df['T_AIR'] / 60
df['T_NON_MOTOR_S'] = df['T_NON_MOTOR'] / 60

In [10]:
# Make car availability binary
df['CAR_AV'] = df['CAR_AV'].replace({9:0})

In [11]:
# Add PT availabilities
df['RAIL_AV'] = (df['T_RAIL']!=inf).astype(int)
df['RAIL_S_AV'] = (df['T_RAIL_S']!=inf).astype(int)
df['RAIL_L_AV'] = (df['T_RAIL_L']!=inf).astype(int)
df['COACH_AV'] = (df['T_COACH']!=inf).astype(int)
df['BUS_AV'] = (df['T_BUS']!=inf).astype(int)
df['ROAD_AV'] = (df['T_ROAD']!=inf).astype(int)
df['AIR_AV'] = (df['T_AIR']!=inf).astype(int)
df['NON_MOTOR_AV'] = (df['C_NON_MOTOR']!=inf).astype(int)

In [12]:
# Merge rail and road PT
df['MODE'] = df['MODE'].replace({2:1, 3:4})

In [13]:
# Remove trips where mode is car but the car availability is zero
# because it irritates the MLE algorithm
mask = ((df['MODE']==6) & (df['CAR_AV']==0))
drop_ratio = len(df.loc[mask])/len(df.loc[df['MODE']==6])
lengths = list(df.loc[mask, 'DIST'])
print('Share of car trips dropped: {}'.format(drop_ratio))
df = df.loc[~mask]
# Drop other modes equally
#for m in [1,4]:
#    n_drops = int(np.round(drop_ratio * len(df.loc[df['MODE']==m])))
#    df = df.drop(df.loc[(df['MODE']==m) & (df['DIST']<max(lengths)) &
#                        (df['DIST']>min(lengths))].sample(n_drops).index)
#    print('Dropped ' + str(n_drops) + ' trips of mode ' + str(m))
print(len(df))

Share of car trips dropped: 0.0141490154593321
93461


In [ ]:
# There is one observation with purpose commuting by air.
# Map it to business trips, because it corrupts the estimation
df.loc[(df['PURPOSE'].isin([1,2]))&(df['MODE']==5), 'PURPOSE'] = 4

In [14]:
# Check that PT trips have no infinity cost
for mode, col in zip([1, 4, 5], ['RAIL_AV', 'ROAD_AV', 'AIR_AV']):
    assert len(df.loc[((df['MODE']==mode) & ~(df[col]))].index) == 0, str(mode)

### Model variables
All columns are variables. DefineVariable creates a new column in the database.

#### Cost damping

Many modelling studies have shown that cost damping is required in order to flatten the tail of time/cost elasticities, i.e. decrease the impact of long distances on choice results to prevent from overestimation of time/cost parameters. Cost damping represents the property of decreasing marginal utility. It is commonly approached with Box-Cox transformations of generalised cost (usually defined as the sum of travel time and travel expenditures divided by the value of time). Daly (2010) proposes a hybrid function as a sum of the linear term and the narural logarithm of the same. However, the linear term still dominates cost on long distances. Rich (2015), main developer of the Danish National Transport Model, proposes a more complex spline function which successfully manages cost damping and even outperforms the Box-Cox transformation in terms of stability of elasticities.

In [15]:
# The cost damping function from Rich (2020)
c = params['estimation']
def spline(x, beta, c1, c2, Q=3):
    alpha = [0, -beta/2*ex.Power(ex.log(c1),3),
             -beta/2*ex.log(c1)*(3*ex.Power(ex.log(c2),2)+ex.Power(ex.log(c1),2))] # for Q=3
    theta = [1, 3/2*ex.log(c1), 3*ex.log(c1)*ex.log(c2)] # for Q=3
    return (beta*theta[0]*ex.Power(ex.log(x),Q-1+1) + alpha[0]) * (x<c1) \
    + (beta*theta[1]*ex.Power(ex.log(x),Q-2+1) + alpha[1]) * (x>=c1)*(x<c2) \
    + (beta*theta[2]*ex.Power(ex.log(x),Q-3+1) + alpha[2]) * (x>=c2)

This functional form performes poorly


#### Generalised cost

Both, travel time and monetary cost should be affected by cost damping measures. It is logical to define a generalised cost term `GC` with dimension of time units. This requires definition or estimation of values of time, in order to rescale monetary units, for all segments. Usually, the value of time (VoT) is distance-dependent. In the case of Germany, VoT can be taken from research undertaken within the Federal Government's transport study "Bundesverkehrswegeplan 2030": Axhausen et al. 2015. Ermittlung von Bewertungsansätzen für Reisezeiten und Zuverlässigkeit auf der Basis eines Modells für modale Verlagerungen im nicht-gewerblichen und gewerblichen Personenverkehr für die Bundesverkehrswegeplanung

In [16]:
# VoT from literature, distance-dependent, see cal19
VoT = pd.read_csv(input_path + 'vot.csv', header=[0,1], index_col=0)
VoT.sample(2)

,root,Fz1,Fz2,Fz3,Fz4,Fz6,root,Fz1,Fz2,Fz3,...,Fz2,Fz3,Fz4,Fz6,root,Fz1,Fz2,Fz3,Fz4,Fz6
,all,all,all,all,all,all,PT,PT,PT,PT,...,air,air,air,air,car,car,car,car,car,car
846,22.6602,21.6434,9.18,11.9,31.2012,16.8786,16.9954,12.8536,13.48,17.2836,...,89.5270,89.5270,241.4638,49.6124,22.3718,21.1758,13.1774,28.1514,25.0290,16.8786
391,16.4744,15.8198,9.18,11.9,18.9440,12.7696,12.5260,9.8722,13.48,14.3040,...,37.8282,37.8282,43.7598,22.8464,16.3662,15.6216,10.0604,20.9718,17.5854,12.7696


In [17]:
# Make distance integer
df['DIST'] = df['DIST'].apply(int)

In [20]:
# Create the initial database and make columns global variables
database = db.Database('MiD2017', df.copy())
globals().update(database.variables)
database.getSampleSize()

93461

### Estimation parameters

In [21]:
asc_rail = ex.Beta('asc_rail', 0, None, None, 0)
asc_rail_s = ex.Beta('asc_rail_s', 0, None, None, 0)
asc_rail_l = ex.Beta('asc_rail_l', 0, None, None, 0)
asc_coach = ex.Beta('asc_coach', 0, None, None, 0)
asc_bus = ex.Beta('asc_bus', 0, None, None, 0)
asc_road = ex.Beta('asc_road', 0, None, None, 0)
asc_air = ex.Beta('asc_air', 0, None, None, 0)
asc_car = ex.Beta('asc_car', 0, None, None, 1)
asc_non_motor = ex.Beta('asc_non_motor', 0, None, None, 0)

In [22]:
b_t = ex.Beta('b_t', 0, None, None, 0)
b_t_s = ex.Beta('b_t_s', 1, None, None, 0)
b_t_rnd = b_t + b_t_s * ex.bioDraws('b_t_rnd', 'NORMAL')
b_c = ex.Beta('b_c', 0, None, None, 0)
b_c_s = ex.Beta('b_c_s', 1, None, None, 0)
b_c_rnd = b_c + b_c_s * ex.bioDraws('b_c_rnd', 'NORMAL')

In [75]:
# generalised cost function parameter
b_gc = ex.Beta('b_gc', 0, None, None, 0)
b_gc_s = ex.Beta('b_gc_s', 1, None, None, 0)
b_gc_rnd = b_gc + b_gc_s * ex.bioDraws('b_gc_rnd', 'NORMAL')

In [25]:
# Parameters for the nested logit structure
mu_pt = ex.Beta('mu_pt', 1, 1, 10, 0)

In [49]:
# Individual parameters (require Mixed Logit formulation)
# We need one parameter for the mean, one for the variance, and one random draw parameter.
# It is advised not to use 0 as starting value for the variance parameter
b_ticket = ex.Beta('b_ticket', 0, None, None, 0)
b_ticket_s = ex.Beta('b_ticket_s', 1, None, None, 0)
b_ticket_rnd = b_ticket + b_ticket_s * ex.bioDraws('b_ticket_rnd', 'UNIFORM_HALTON2')
b_inc = ex.Beta('b_inc', 0, None, None, 0)
b_inc_s = ex.Beta('b_inc_s', 1, None, None, 0)
b_inc_rnd = b_inc + b_inc_s * ex.bioDraws('b_inc_rnd', 'UNIFORM_HALTON2')
b_cars = ex.Beta('b_cars', 0, None, None, 0)
b_cars_s = ex.Beta('b_cars_s', 1, None, None, 0)
b_cars_rnd = b_cars + b_cars_s * ex.bioDraws('b_cars_rnd', 'UNIFORM_HALTON2')

### Utility functions

In [86]:
# Model specification with spline function transformation - disaggregated
p_dict_rev = dict(zip(p_dict.values(), p_dict.keys()))
car = '_car' # take the knot points for car available
V_RAIL = {p: asc_rail
              + C_RAIL * b_c
              + spline(T_RAIL, b_t, c['c1_time_'+p_dict_rev[p]+car]*60, c['c2_time_'+p_dict_rev[p]+car]*60)
          for p in df['PURPOSE'].unique()}
V_ROAD = {p: asc_road
              + C_ROAD * b_c
              + spline(T_ROAD, b_t, c['c1_time_'+p_dict_rev[p]+car]*60, c['c2_time_'+p_dict_rev[p]+car]*60)
          for p in df['PURPOSE'].unique()}
V_AIR = {p: asc_air
             + C_AIR * b_c
             + spline(T_AIR, b_t, c['c1_time_'+p_dict_rev[p]+car]*60, c['c2_time_'+p_dict_rev[p]+car]*60)
         for p in df['PURPOSE'].unique()}
V_CAR = {p: asc_car
             + C_CAR * b_c
             + spline(T_CAR, b_t, c['c1_time_'+p_dict_rev[p]+car]*60, c['c2_time_'+p_dict_rev[p]+car]*60)
         for p in df['PURPOSE'].unique()}
V_NON_MOTOR = {p: asc_non_motor 
                   + spline(T_NON_MOTOR, b_t, c['c1_time_'+p_dict_rev[p]+car]*60, c['c2_time_'+p_dict_rev[p]+car]*60)
               for p in df['PURPOSE'].unique()}

# Model specification for business trips
V_RAIL[2] = asc_rail + spline(T_RAIL, b_t, c['c1_time_business'+car]*60, c['c2_time_business'+car]*60)
V_ROAD[2] = asc_road + spline(T_ROAD, b_t, c['c1_time_business'+car]*60, c['c2_time_business'+car]*60)
V_AIR[2] = asc_air + spline(T_AIR, b_t, c['c1_time_business'+car]*60, c['c2_time_business'+car]*60)
V_CAR[2] = asc_car + spline(T_CAR, b_t, c['c1_time_business'+car]*60, c['c2_time_business'+car]*60)
V_NON_MOTOR[2] = asc_non_motor + spline(T_NON_MOTOR, b_t, c['c1_time_business'+car]*60, c['c2_time_business'+car]*60)

### Run the estimation

In [30]:
# Define level of verbosity
logger = message.bioMessage()
#logger.setSilent()
logger.setWarning()
#logger.setGeneral()
#logger.setDetailed()

In [87]:
# Map modes to utility functions
V = {1:V_RAIL,
     4:V_ROAD,
     5:V_AIR,
     6:V_CAR,
     7:V_NON_MOTOR}

In [32]:
# Map the availability of alternatives with MODE as key
av = {1:RAIL_AV,
      4:ROAD_AV,
      5:AIR_AV,
      6:CAR_AV,
      7:NON_MOTOR_AV}

In [33]:
# Mode nests as tuples with nest name and dictionary where
# alternative IDs are mapped to alpha values. Missing ID's alpha is zero
# Alternatively use lists with mode ID without alpha
pt = mu_pt, [1, 4]#[1, 2, 3, 4]
air = 1, [5]
car = 1, [6]
walk = 1, [7]
nests = pt, air, car, walk

In [104]:
# Write results to an Excel file
writer = pd.ExcelWriter(input_path + 'estimation_results_car_own_ML.xls', engine='xlsxwriter')

In [89]:
# Choose the multinomial logit model
mnl = models.logit(V, av, MODE)

In [90]:
# Monte Carlo Integration
mnl_mix = ex.log(ex.MonteCarlo(mnl))

In [91]:
database = db.Database('MiD2017', df.copy())
#database.remove(PURPOSE!=1)
model_mnl = bio.BIOGEME(database, mnl_mix, numberOfDraws=5000)
model_mnl.modelName = 'MNL'

In [92]:
results_mnl = model_mnl.estimate()

In [93]:
results = results_mnl.getEstimatedParameters()
for key, val in results_mnl.getGeneralStatistics().items():
    results.loc[key] = [val[0], val[1]] + ['' for i in range(len(results.columns)-2)]
results

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_air,-5.66016,0.198802,-28.4713,0,0.243355,-23.2589,0
asc_non_motor,-4.62377,0.0432325,-106.951,0,0.0450704,-102.59,0
asc_rail,-2.15596,0.0152572,-141.308,0,0.0172081,-125.288,0
asc_road,-3.10958,0.0186338,-166.879,0,0.0188077,-165.336,0
b_gc,-0.432392,0.0120719,-35.8182,0,0.0175303,-24.6655,0
b_gc_s,0.16492,0.00534774,30.8392,0,0.00703688,23.4365,0
Number of estimated parameters,6,,,,,,
Sample size,93461,,,,,,
Excluded observations,0,,,,,,
Init log likelihood,-127815,.7g,,,,,


In [94]:
results.to_excel(writer, sheet_name=model_mnl.modelName)

In [95]:
# Choose the logarithmic nested logit model
nl = models.lognested(V, av, nests, MODE)
# Monte Carlo Integration
nl_mix = ex.log(ex.MonteCarlo(nl))

In [96]:
# Nested Logit
model_nl = bio.BIOGEME(database, nl_mix, numberOfDraws=1000)
model_nl.modelName = 'NL'

NameError: name 'nl_mix' is not defined

In [ ]:
results_nl = model_nl.estimate()

In [ ]:
results = results_nl.getEstimatedParameters()
for key, val in results_nl.getGeneralStatistics().items():
    results.loc[key] = [val[0], val[1]] + ['' for i in range(len(results.columns)-2)]
results

In [ ]:
results.to_excel(writer, sheet_name=model_nl.modelName)

In [100]:
# Run the estimation by purpose
results = []
for p in [1,2,3,4,6]:
    for car in [0,1]:
        database = db.Database('MiD2017', df.copy())
        database.remove(PURPOSE!=p)
        database.remove(CAR_AV!=car_ownership)
        print('Sample size for purpose {}, car_ownership {}: {}'.format(
            p, car, database.getSampleSize()))
        mnl = models.logit(V, av, MODE)
        mnl_mix = ex.log(ex.MonteCarlo(mnl))
        model = bio.BIOGEME(database, mnl_mix, numberOfDraws=5000) # Choose the model formulation
        model.modelName = str(p) + '_' + str(car) # Name it
        results.append(model.estimate()) # Estimation
        output = results[-1].getEstimatedParameters()
        # Add results to the Excel file
        for key, val in results[-1].getGeneralStatistics().items():
            output.loc[key] = [val[0], val[1]] + ['' for i in range(len(output.columns)-2)]
        output.to_excel(writer, sheet_name=model.modelName)

Sample size for purpose 1, car_ownership 0: 644
Sample size for purpose 1, car_ownership 1: 24620
Sample size for purpose 2, car_ownership 0: 117


/home/local/RL-INSTITUT/marlin.arnz/quetzal/lib/python3.6/site-packages/biogeme/optimization.py:313: RuntimeWarning: overflow encountered in double_scalars
  else np.inf for i in range(self.n)])


Sample size for purpose 2, car_ownership 1: 3464
Sample size for purpose 3, car_ownership 0: 400
Sample size for purpose 3, car_ownership 1: 22519
Sample size for purpose 4, car_ownership 0: 5611
Sample size for purpose 4, car_ownership 1: 5611
Sample size for purpose 6, car_ownership 0: 872
Sample size for purpose 6, car_ownership 1: 35214


In [106]:
writer.save()